#1. Ejemplo del método de clasificación Naive Bayes
#Crop mapping using fused optical-radar data set Data Set

Este gran conjunto de datos es un radar óptico bitemporal fusionado para la clasificación de tierras de cultivo. Las imágenes fueron recolectadas por satélites RapidEye (ópticos) y el sistema de radar de apertura sintética de vehículos aéreos no tripulados (UAVSAR) (Radar) sobre una región agrícola cerca de Winnipeg, Manitoba, Canadá en 2012.
Hay 2 * 49 funciones de radar y 2 * 38 funciones ópticas para dos fechas: 05 y 14 de julio de 2012.

Existen siete clases de tipos de cultivos para este conjunto de datos de la siguiente manera: 1-Maíz; 2-guisantes; 3- Canola; 4-soja; 5- Avena; 6- Trigo; y 7 de hoja ancha.

1.   Corn
2.   Peas;
3.Canola
4.Soybeans
5.Oats
6.Wheat
7.Broadleaf



Más: https://archive.ics.uci.edu/ml/datasets/Crop+mapping+using+fused+optical-radar+data+set





#2. Importación de librerías y montando google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as ______
import pandas as ______
import ______ as plt
import seaborn as sns
import itertools

In [ ]:
from sklearn.preprocessing import ______ #Codificación de etiquetas
from sklearn.preprocessing import ______, StandardScaler #Escaladores
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import ______ #Análisis de componentes principales
from sklearn.ensemble import RandomForestClassifier

In [ ]:
path = r'_______'
dataset_path = os.path.join(path,'WinnipegDataset.txt')

# 3. Lectura y visualuzación del dataset

In [ ]:
dataset = pd.______(dataset_path)
dataset.dropna(axis=0, inplace=True)
dataset.drop_duplicates()
dataset.head()

In [ ]:
dataset.shape

In [ ]:
print ('Conteo de valores faltantes: \n')
print (dataset.______().sum().sum())

Distribución de clases

In [ ]:
fig, ax =plt.subplots(figsize=(16,5))

______.countplot('label', data=dataset)
plt.title('Distribution of Classes Dataset')
plt.xlabel('Cropland Class', fontsize=14)
plt.ylabel('Count', fontsize=14)

plt.show()

In [ ]:
class_distr = pd.Series(100 * dataset.label.value_counts(normalize=True)).sort_index()
print('Distribución de clases \n')
for i in range(0,7):
    print(f'Tipo de cultivo {class_distr.index[i]}: {class_distr.iloc[i]:.2f} %')

#3. Escalamiento y codificación de etiquetas

In [ ]:
scaler = ______()
dataset.loc[:, dataset.columns != 'label'] = scaler.fit_transform(dataset.loc[:, dataset.columns != 'label'])

In [ ]:
le = ______()
dataset['label'] = le.fit_transform(dataset.label.values)
dataset.head()

#4.  Selección de características (elegir uno)

In [ ]:
#Feature selection
fs = SelectKBest(score_func=f_classif, k=______) #default 10 features
np_X = dataset.iloc[:, 1:].to_numpy()
np_Y = dataset.iloc[:, 0].to_numpy()
X_selec = fs.fit_transform(np_X, np_Y)
attr_names = dataset.columns.values.tolist()
# Lista de bools que indican si es un atributo seleccionado o no
mask = fs.get_support()
# Lista de los K mejores atributos
new_features = [] 
for bool, feature in zip(mask, attr_names):
    if bool:
        new_features.append(feature)
dataset_FS = pd.DataFrame(X_selec, columns=new_features)
dataset_FS['label'] = dataset['label']

dataset_FS.head()

In [ ]:
#Componentes principales
pca = PCA(______)
pc = pca.fit_transform(dataset.iloc[:,1:])
dataset_pca = pd.DataFrame(data = pc, columns=range(pc.shape[1]))
dataset_pca = pd.concat([dataset.iloc[:,0], dataset_pca],  axis = 1)

print('Número de componentes principales: %s'%len(pca.explained_variance_ratio_))
print('Varianza acumulada con %s componentes: %s'%(len(pca.explained_variance_ratio_),np.sum(pca.explained_variance_ratio_)))

dataset_pca.head()

#5. Dividiendo el conjunto en entrenamiento y prueba

In [ ]:
X = np.array(______.iloc[:,1:])
y = np.array(______['label'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = ______, random_state = 0)

print(f'Dimension de los datos de entrenamiento {X_train.shape}')
print(f'Dimension de los datos de prueba {X_test.shape}')

#6. Entrenando y prediciendo con CART

In [ ]:
dectree = DecisionTreeClassifier(max_depth= 10) 
dectree = dectree.______(X_train,y_train)

In [ ]:
y_pred = dectree.______(X_test)
score = metrics.accuracy_score(y_test, y_pred)
print("Test Acc: %s"%score)

In [ ]:
#Funciones para MC y métricas
def plot_confusion_matrix(cm, classes, tit, normalize=False):
    if normalize:
        cm = cm.astype('float')/cm.sum(axis=1)
        title, fmt = 'Matriz de confusión normalizada', '.2f'
    else:
        title, fmt = tit, 'd'
    plt.figure(figsize=(10,8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)#, fontsize=12)
    plt.colorbar(pad=0.05)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=40)
    plt.yticks(tick_marks, classes)
    thresh = cm.max()/2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),horizontalalignment="center", 
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('Clase Verdadera')#, fontsize=10)
    plt.xlabel('Clase Predicha')#, fontsize=10)
    plt.savefig(title+'.png')
    #plt.grid(b=None)
    plt.show()
def sens_spec(cls_names, y_true, y_pred, pesos):
  sensitivity = []
  specificity = []
  prec=[]
  fscore = []
  sd = []
  for i,name in enumerate(cls_names):
    TP = np.sum((y_true==name) & (y_pred==name))
    TN = np.sum((y_true!=name) & (y_pred!=name))
    FP = np.sum((y_true!=name) & (y_pred==name))
    FN = np.sum((y_true==name) & (y_pred!=name))
    sensitivity.append(TP/(TP+FN))
    specificity.append(FP/(TN+FP))
    prec.append(TP/(TP+FP))
    fscore.append(2*(prec[i]*sensitivity[i])/(prec[i]+sensitivity[i]))
  sensitivity.append(sum([x*y for x,y in zip(sensitivity,pesos)]))
  specificity.append(sum([x*y for x,y in zip(specificity,pesos)]))
  prec.append(sum([x*y for x,y in zip(prec,pesos)]))
  fscore.append(sum([x*y for x,y in zip(fscore,pesos)]))
  for i in range(len(cls_names)):
    sd.append(np.sqrt((sensitivity[i]-sensitivity[-1])**2))
  sd.append(np.nan)
  return sensitivity, specificity, prec, fscore, sd

In [ ]:
clases = dataset['label'].unique().tolist()
n_clases = len(clases)
pesos = dataset['label'].value_counts().sort_index().tolist()/ np.sum(dataset['label'].value_counts().tolist())

In [ ]:
predictions = np.float32(y_pred)
true_labels = np.float32(y_test)
cnf_matrix = confusion_matrix(true_labels, predictions, labels=range(n_clases))
tit = 'Matriz de confusión árbol de decisión (CART)'
plot_confusion_matrix(cnf_matrix,clases, tit, normalize=False)

In [ ]:
sensitivity, FPR, prec, fscore, sd = sens_spec(range(n_clases), true_labels, predictions, pesos)
d = {'TPR':sensitivity, 'FPR':FPR, 'Acc':prec, 'F-score':fscore}
ind = clases+['Promedio']
df = pd.DataFrame(d, index=ind)
index = df.index
precprom = sum([x*y for x,y in zip(prec,pesos)])
index.name = 'Acc Pond: %s'%precprom
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(df, annot=True)

#7. Entrenando y prediciendo con Random Forest

In [ ]:
random_forest_classifier = RandomForestClassifier(n_estimators=100, n_jobs=4)

random_forest_classifier.______(X_train, y_train)

In [ ]:
y_pred_rf = random_forest_classifier.______(X_test)
score = metrics.accuracy_score(y_test, y_pred_rf)
print("Test Acc: %s"%score)

In [ ]:
predictions = np.float32(y_pred_rf)
true_labels = np.float32(y_test)
cnf_matrix = confusion_matrix(true_labels, predictions, labels=range(n_clases))
tit = 'Matriz de confusión Random Forest'
plot_confusion_matrix(cnf_matrix, clases, tit, normalize=False)

In [ ]:
sensitivity, FPR, prec, fscore, sd = sens_spec(range(n_clases), true_labels, predictions, pesos)
d = {'TPR':sensitivity, 'FPR':FPR, 'Acc':prec, 'F-score':fscore}
ind = clases+['Promedio']
df = pd.DataFrame(d, index=ind)#.transpose()
index = df.index
precprom = sum([x*y for x,y in zip(prec,pesos)])
index.name = 'Acc Pond: %s'%precprom
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(df, annot=True)